## Corpus Linguistics - LING3038/6038
### Week 3

How can we do some descriptive corpus statistics with R?

An easy way to get started is to import a text from the web. 

In [ ]:
moby<-read.delim("https://www.gutenberg.org/files/2701/2701-0.txt",encoding="UTF-8",sep="\n") #this function has 3 arguments
#the website from which we are reading data
#the text encoding. UTF-8 works well for most languages. If the text was encoded with a different system, we could specfic this here
#what is used to separate parts of the text in the file. Here it is different lines so we use \n to mean 'new line'
moby2

In [ ]:
#we can see the first and last bits of our object by using head() and tail(). 
#We use a comma and then an argument to define how many lines we want to see.

head(moby,300)
tail(moby,300)

There is a lot of header/footer information that we do not want to contribute to our word counts. 
We want to exlude that header/footer information. How can we do that?
We can figure out the number of the rows where the data starts and ends.
Then we save that portion as a new object.

In [ ]:
#find the line that starts the actual text
moby[14:16,]


#find the line that ends the actual text
moby[18928:18929,]

This works if you already know the line numbers of around the start/end of the text


In [ ]:
moby.text<-moby[15:18928,]

In [ ]:
##Another way to read in texts is with the funtcion readLines()
##Then name our lines based on finding certain indicators/signals of when the text starts and stops
##to find those indications we search for them using grep()

moby2<-readLines("https://www.gutenberg.org/files/2701/2701-0.txt",encoding="UTF-8")
head(moby2)

##Then we search for strings that signal the start/stop of the text and name them:
##Most Gutenberg texts contain a string like "START OF xxxx" / "END OF xxxx"

# the text starts after the Project Gutenberg header...
start <- grep("START OF THE PROJECT GUTENBERG", moby2, value=F) + 1 #one line after the line with this text

# ...end ends at the Project Gutenberg footer.
stop <- grep("END OF THE PROJECT GUTENBERG", moby2,value=F) - 1 #one line before the line with this text

start
stop



In [ ]:
moby.lines <- moby2[start:stop]
head(moby.lines,25) #lets see the start of the text now

We might also decide that we do not want to start with "CHAPTER 1" but rather with the classic line "Call me Ishmael". How would we change our start search to make this happen?

In [ ]:
#You put in what we want to search for (Call me Ishmael)
Ishmael.start <- grep("", moby2, value=F) #no plus 1 as we want to start with this line of text

#now create another object called moby.lines2. You put in the start and stop objects

moby.lines2<-moby2[:]

head(moby.lines2,20)

Note that these different strategies of reading in and selecting our portions of the text results in different sized objects.


In [ ]:
length(moby.text)
length(moby.lines)
length(moby.lines2)



Now let's work with our object 'moby.lines'

In [ ]:
head(moby.lines)

In [ ]:
#we now are going to turn everything into lowercase

moby.text.lowercase<-tolower(moby.lines)
head(moby.text.lowercase)

In [ ]:
#Right now our ojbect is a list of character strings of different lengths. We want to make our text into one long string.
#We do this with paste() and unlist()

moby.text.lowercase.vector<-paste(unlist(moby.text.lowercase), collapse =" ")

length(moby.text.lowercase)
length(moby.text.lowercase.vector)

In [ ]:
moby.vec<-moby.text.lowercase.vector #this is just to create a back-up object and work with an object with a shorter name

There are a lot of R packages out there that deal with corpus data and have functions to do the main corpus things like turn a text into a collection of word tokens. Here are a few:

In [ ]:
library(corpus)
moby.toks.corpus<-text_tokens(moby.vec, drop_punct=T)
moby.toks.corpus

In [ ]:
library(stringr)
moby.toks.stringr<-str_split(moby.vec, "\\s|[[:PUNCT:]]") #this is a regular expression. we'll get to these soon
moby.toks.stringr

In [ ]:
library(tm)
moby.toks.tm.scan<-scan_tokenizer(moby.vec) #this package has a few different tokenizers with different function names
moby.toks.tm.scan

Now that we have a bunch of tokens, we can start counting them, their types and doing basic corpus description tasks. I am going to load {data.table} to do this and turn my {corpus} created tokens into a data.table

In [ ]:
library(data.table)
dt<-as.data.table(moby.toks.corpus, na.rm=T)
dt

In [ ]:
##turn it into a frequency table
#add a column to count the words
dt[, word_count := .N, by = V1] #.N counts the instances of things##turn it into a frequency table
#add a column to count the words
dt[, word_count := .N, by = V1] #.N counts the instances of things

In [ ]:
#reduce tokens to types by getting the unique tokens
dtm<-unique(dt[,c("V1","word_count")])
colnames(dtm)[1]<-"word"
colnames(dtm)[2]<-"freq"

In [ ]:
dtm

In [ ]:
#Let's sort this in a more useful way
setkey(dtm,freq)
dtm


In [ ]:
library(ggplot2)
ggplot(subset(dtm, freq > 350) ,aes(reorder(word, -freq), freq))+
  geom_bar(stat="identity")+
  theme(axis.text.x=element_text(angle=90, hjust=1)) + ggtitle("Moby Dick word frequency (over 350 tokens)")+
  xlab("Word")+
  ylab("Frequency")


In [ ]:
#there are a lot of "stopwords" here. let's get rid of some.
library(stopwords)
head(stopwords_en,20)
length(stopwords_en)

dtm2<-dtm[! dtm$word %in% stopwords_en,] 
dtm2

In [ ]:
ggplot(subset(dtm2, freq > 200) ,aes(reorder(word, -freq), freq))+
  geom_bar(stat="identity")+
  theme(axis.text.x=element_text(angle=90, hjust=1)) + ggtitle("Moby Dick word frequency (over 200 tokens, stopwords removed)")+
  xlab("Word")+
  ylab("Frequency")

In [ ]:
##Histograms
dtm3 <- dtm2[order(-freq)] #this also orders like setkey(dtm,freq) above
dtm3[, rank:= .I, by="freq"] #we are adding a column for the frequency rank
dtm3

In [ ]:
a<-ggplot(dtm3 ,aes(rank,freq))+
  geom_point(position=position_jitter(width=.05,height=.05))+ 
  ggtitle("Moby Dick word rank v frequency")+
  xlab("Rank")+
  ylab("Frequency")

In [ ]:
d<-ggplot(dtm3 ,aes(rank,freq))+
  geom_point(position=position_jitter(width=.05,height=.05))+ 
  scale_x_log10()+
  scale_y_log10()+
  ggtitle("Moby Dick log-log plot")+
  xlab("Rank (on log scale)")+
  ylab("Frequency (on log scale)")

In [ ]:
library(gridExtra)
grid.arrange(a,d, ncol = 2)

In [ ]:
##word cloud

library(wordcloud2)
dtm4<-dtm3[dtm3$freq >10,]
wordcloud2(data=dtm4, size = 1, color="random-dark", shape="triangle", hoverFunction=NULL)
#if the size is too big, sometimes the most frequent word won't show up